In [12]:
import org.apache.spark.streaming.StreamingContext._
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SQLContext
import org.apache.spark.streaming.Duration
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.types
import spark.implicits._

import org.apache.spark.streaming.StreamingContext._
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SQLContext
import org.apache.spark.streaming.Duration
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.types
import spark.implicits._


In [13]:
!rm -r checkpoint


In [14]:
!mkdir checkpoint

In [15]:
val schema = spark.read.json("input/*.json").schema

schema: org.apache.spark.sql.types.StructType = StructType(StructField(agent-phone,StringType,true),StructField(bath,StringType,true),StructField(bedrooms,StringType,true),StructField(beds,StringType,true),StructField(data-est,StringType,true),StructField(data-lk,StringType,true),StructField(data-pk,StringType,true),StructField(data-pos,StringType,true),StructField(full_bathrooms,StringType,true),StructField(home_type,StringType,true),StructField(ldp-description-text,StringType,true),StructField(ldp-phone-link,StringType,true),StructField(price,StringType,true),StructField(property-info-address-citystatezip,StringType,true),StructField(property-info-address-main,StringType,true),StructField(sold date,StringType,true),StructField(sqft,StringType,true),StructField(status,StringType,true),...


In [16]:
val df = spark.readStream
    .format("json")
    .schema(schema)
    .option("multiLine", "false")
    .load("input/*.json")
    .select("*", "_metadata")

df: org.apache.spark.sql.DataFrame = [agent-phone: string, bath: string ... 18 more fields]


/*
val df = spark.readStream
    .format("json")
    .schema(schema)
    .option("multiLine", "false")
    .option("cleanSource", "archive")
    .option("sourceArchiveDir", f"input_archive")
    .load("input/*.json")
    .select("*", "_metadata")
*/

In [17]:
val df2 = df.writeStream
    .option("checkpointLocation", "checkpoint")
    .option("overwriteSchema",true)
    .trigger(org.apache.spark.sql.streaming.Trigger.ProcessingTime("1 second"))
    .toTable("bronze_houses")

df2: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4a51264b


In [ ]:
df2.processAllAvailable
//df2.stop
df2.awaitTermination

In [ ]:
spark.catalog.listTables show

In [ ]:
val df3 = spark.read.format("jdbc").table("bronze_houses")

In [ ]:
df3.count

In [ ]:
df2.stop

In [ ]:
df3.select("_metadata").distinct.show(truncate=false)